In [209]:
from sys import version_info
version_info

def print_util(func):
    # print object instance / function result
    def util(*a,**kw):
        result = func(*a,**kw)
        print(a[-1],result)
        return result
    return util

# Process of instance creation

difference between \__new__ and \__init__


In [204]:
class Foo:
    def __new__(cls,*a,**kw):
        print("{} allocated __new__ is called".format(cls.__name__))
        obj = super().__new__(cls) 
        # used when u need to hook into obj creation - immuatable 
        obj.d = "assignment can happen in __new__"
        print(id(obj))
        return obj
    
    # object created from new passed to __init__ as self
    def __init__(self,*a,**kw):
        print(id(self))
        print("id of object from new is the same as self")
        
        # self has no __name__. __name__ belongs to class
        cls = self.__class__  
        
        # instance object dir look up is from class
        # pass __dir__ in new instance __dir__ cannot be change
        # likewise __call__ and __len__ is attached to class
        new_dir = self.__class__.__dir__
        
        print("__dir__ is a {} object".format(type(new_dir)))
        print("Is __dir__ a dict".format(isinstance(new_dir,dict)))
        self.__class__.__dir__ = { str(i):j for i,j in zip(range(len(a)),a)}
        print("{0} initialisation after __new__, __init__ called".format(cls.__name__))
        
a = Foo(1,2,3)

Foo allocated __new__ is called
2177923459504
2177923459504
id of object from new is the same as self
__dir__ is a <class 'method_descriptor'> object
Is __dir__ a dict
Foo initialisation after __new__, __init__ called


# Using New vs Init

- \__init__ returns None
- so it means that object is created before init
- what if you want to return a int object 
- during object creation, pass in int

In [213]:
class PositiveInteger(int): 
    @print_util
    def __new__(cls, a):
        if a > 0:
            return super().__new__(cls,a)
        else:
            raise ValueError

PositiveInteger(34.545)
PositiveInteger(3.545)
#PositiveInteger(-3)
type(PositiveInteger)

34.545 34
3.545 3


type

# Metaclasses and object model in Python

- type(instance) == class
- type(class) ==? -- metaclass
- from above type(PositiveInterger) == type
- what is metaclass? -- creation of class


## Application of metaclassing

Base class enforcing constraint on derived class

eg. Bar(Foo) and Foo wants to enforce something on Bar

- Derive and Base both will invoke metaclass
- Contraint in metaclass propagate to subclasses

In [222]:
from collections import Iterable
# Sometimes you just need to iterate through sth 
# - be it a list / tuple / dict

print(isinstance([],Iterable))
print((isinstance((),Iterable)))
print((isinstance({},Iterable)))

type([]), type({}), type(()), type(Iterable)

# dict tuple list are instances of iterable?

True
True
True


(list, dict, tuple, abc.ABCMeta)

In [225]:
class metaclass(type):
    def __new__(cls, *a):
        return type.__new__(cls,*a)
    
    def __init__(self,name,base,body):
        if name == "Derived":
            print("Hooked into Derived during class creation")
        
    
class Base(metaclass=metaclass):
    pass

class Derived(Base):
    pass

Hooked into Derived during class creation
